### Demo - Deleted delta Stores

In [1]:
/* 
    ColumnStores are READONLY, so when we delete (or update rows), it simulates R/W by writing a delete followed by an insert into the delta store, and merging this at query time.

    This demo shows impact of Updatees and how we can see the deleted rows in the delta store.
*/

	UPDATE [dbo].[FactFinance100m]
	SET Amount=Amount+1
	where DateKey =20131228 


(64758 rows affected)

Total execution time: 00:00:03.717

In [3]:
/* we can see the impact on the deleted/inserted rows and open Delta Stores */

SELECT * FROM dbo.vColumnstoreStats WHERE table_name='FactFinance100m'


(1 row affected)

Total execution time: 00:00:01.221

execution_date,database_name,schema_name,table_name,partition_number,partition_scheme,object_id,index_name,row_count,deleted_row_count,row_group_count,compressed_row_count,compressed_rowgroup_count,open_rowgroup_count,open_row_count,compressed_row_max,compressed_row_avg,fragmentation_density,fragmentation_deletes,fragmentation_open
2022-07-08 13:12:32.560,AdventureWorksDW,dbo,FactFinance100m,NULL,NULL,162867697,ClusteredIndex_bfe60ded6e5d4ff498d34447e5d3f9bf,86064758,64758,239,86000000,180,59,64758,1000532,477777,54.4400,0.1100,0.1100


A simple REORGANIZE wil not close the open row groups.

While a <span style="color: rgb(0, 128, 0); font-family: Consolas, &quot;Courier New&quot;, monospace; font-size: 12px; white-space: pre;">REORGANIZE&nbsp;WITH&nbsp;(COMPRESS_ALL_ROW_GROUPS&nbsp;=&nbsp;ON)</span> will close the open RowGroup for inserts it will also not take care of deleted row. See below - after REGORG we still have the deleted rows  


In [6]:
exec   [dbo].[ColumnstoreOptimize]  @Tables='FactFinance100m'
,@DensityThreshold=null
,@OpenThreshold=10000
,@DeleteThreshold=null
,@TimeLimit =null
, @Execute='Y'

SELECT * FROM dbo.vColumnstoreStats WHERE table_name='FactFinance100m'

Date and time: 2022-07-08 13:13:36

Server: aw-dev

Version: 10.0.15665.0

Edition: SQL Azure

Parameters: @Tables= 'FactFinance100m', @DensityThreshold = 25.00, @OpenThreshold = 10000, @DeleteThreshold = 200000, @Execute = Y

...

Date and time: 2022-07-08 13:13:39

SqlCommand: ALTER INDEX ClusteredIndex_bfe60ded6e5d4ff498d34447e5d3f9bf ON [dbo].[FactFinance100m] REORGANIZE WITH (COMPRESS_ALL_ROW_GROUPS = ON)

(1 row affected)

Total execution time: 00:00:05.651

execution_date,database_name,schema_name,table_name,partition_number,partition_scheme,object_id,index_name,row_count,deleted_row_count,row_group_count,compressed_row_count,compressed_rowgroup_count,open_rowgroup_count,open_row_count,compressed_row_max,compressed_row_avg,fragmentation_density,fragmentation_deletes,fragmentation_open
2022-07-08 13:13:40.537,AdventureWorksDW,dbo,FactFinance100m,NULL,NULL,162867697,ClusteredIndex_bfe60ded6e5d4ff498d34447e5d3f9bf,86044901,44901,179,86044901,179,0,0,1000532,480697,54.1600,0.0700,0.0000


Using the ColumnStoreOrganise we can pass a DeleteThrehsold and force a REBUILD, which is an offline operation

(queries blocked until finished)

In [7]:
exec   [dbo].[ColumnstoreOptimize]  @Tables='FactFinance100m'
,@DensityThreshold=null
,@OpenThreshold=null
,@DeleteThreshold=10000
,@TimeLimit =null
,@Execute=null

SELECT * FROM dbo.vColumnstoreStats WHERE table_name='FactFinance100m'

Date and time: 2022-07-08 13:14:00

Server: aw-dev

Version: 10.0.15665.0

Edition: SQL Azure

Parameters: @Tables= 'FactFinance100m', @DensityThreshold = 25.00, @OpenThreshold = 200000, @DeleteThreshold = 10000, @Execute = Y

...

Date and time: 2022-07-08 13:14:03

SqlCommand: ALTER INDEX ClusteredIndex_bfe60ded6e5d4ff498d34447e5d3f9bf ON [dbo].[FactFinance100m] REBUILD

(1 row affected)

Total execution time: 00:00:11.818

execution_date,database_name,schema_name,table_name,partition_number,partition_scheme,object_id,index_name,row_count,deleted_row_count,row_group_count,compressed_row_count,compressed_rowgroup_count,open_rowgroup_count,open_row_count,compressed_row_max,compressed_row_avg,fragmentation_density,fragmentation_deletes,fragmentation_open
2022-07-08 13:14:10.910,AdventureWorksDW,dbo,FactFinance100m,NULL,NULL,162867697,ClusteredIndex_bfe60ded6e5d4ff498d34447e5d3f9bf,86000000,0,180,86000000,180,0,0,1048576,477777,54.4400,0.0000,0.0000
